In [1]:

import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 1
event_codes = [5,6,9,10,13,14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto', verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,baseline=None, preload=True)



Using default location ~/mne_data for EEGBCI...
[........................................] 100.00000 \ (  2.5 MB /   2.5 MB,   2.3 MB/s)   
Do you want to set the path:
    /home/fred/mne_data
as the default EEGBCI dataset path in the mne-python config [y]/n? y
[........................................] 100.00000 / (  2.5 MB /   2.5 MB,  15.0 MB/s)   
[........................................] 100.00000 \ (  2.5 MB /   2.5 MB,  30.2 MB/s)   
[........................................] 100.00000 - (  2.5 MB /   2.5 MB,  18.7 MB/s)   
[........................................] 100.00000 \ (  2.5 MB /   2.5 MB,  25.3 MB/s)   
[........................................] 100.00000 | (  2.5 MB /   2.5 MB,  25.5 MB/s)   
Removing orphaned offset at the beginning of the file.
179 events found
Events id: [1 2 3]
90 matching events found
Loading data for 90 events and 497 original time points ...
0 bad epochs dropped


In [2]:
import numpy as np
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

In [5]:
from braindecode.datautil.signal_target import SignalAndTarget

train_set = SignalAndTarget(X[:60], y=y[:60])
test_set = SignalAndTarget(X[60:], y=y[60:])

In [6]:

from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2],
                        final_conv_length='auto').create_network()
if cuda:
    model.cuda()

In [7]:
from torch import optim

optimizer = optim.Adam(model.parameters())

In [8]:
from braindecode.torch_ext.util import np_to_var, var_to_np
from braindecode.datautil.iterators import get_balanced_batches
import torch.nn.functional as F
from numpy.random import RandomState
rng = RandomState()
for i_epoch in range(30):
    i_trials_in_batch = get_balanced_batches(len(train_set.X), rng, shuffle=True,
                                            batch_size=30)
    # Set model to training mode
    model.train()
    for i_trials in i_trials_in_batch:
        # Have to add empty fourth dimension to X
        batch_X = train_set.X[i_trials][:,:,:,None]
        batch_y = train_set.y[i_trials]
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        # Compute outputs of the network
        outputs = model(net_in)
        # Compute the loss
        loss = F.nll_loss(outputs, net_target)
        # Do the backpropagation
        loss.backward()
        # Update parameters with the optimizer
        optimizer.step()

    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set), ('Test', test_set)):
        # Here, we will use the entire dataset at once, which is still possible
        # for such smaller datasets. Otherwise we would have to use batches.
        net_in = np_to_var(dataset.X[:,:,:,None])
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(dataset.y)
        if cuda:
            net_target = net_target.cuda()
        outputs = model(net_in)
        loss = F.nll_loss(outputs, net_target)
        print("{:6s} Loss: {:.5f}".format(
            setname, float(var_to_np(loss))))
        predicted_labels = np.argmax(var_to_np(outputs), axis=1)
        accuracy = np.mean(dataset.y  == predicted_labels)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))

Epoch 0
Train  Loss: 1.17760
Train  Accuracy: 51.7%
Test   Loss: 1.26024
Test   Accuracy: 53.3%
Epoch 1
Train  Loss: 0.70688
Train  Accuracy: 63.3%
Test   Loss: 0.93679
Test   Accuracy: 56.7%
Epoch 2
Train  Loss: 0.39426
Train  Accuracy: 86.7%
Test   Loss: 0.65984
Test   Accuracy: 66.7%
Epoch 3
Train  Loss: 0.33593
Train  Accuracy: 90.0%
Test   Loss: 0.65626
Test   Accuracy: 63.3%
Epoch 4
Train  Loss: 0.27905
Train  Accuracy: 96.7%
Test   Loss: 0.61044
Test   Accuracy: 56.7%
Epoch 5
Train  Loss: 0.27319
Train  Accuracy: 96.7%
Test   Loss: 0.59499
Test   Accuracy: 66.7%
Epoch 6
Train  Loss: 0.26624
Train  Accuracy: 88.3%
Test   Loss: 0.61360
Test   Accuracy: 70.0%
Epoch 7
Train  Loss: 0.23338
Train  Accuracy: 91.7%
Test   Loss: 0.65626
Test   Accuracy: 73.3%
Epoch 8
Train  Loss: 0.21903
Train  Accuracy: 90.0%
Test   Loss: 0.68762
Test   Accuracy: 63.3%
Epoch 9
Train  Loss: 0.18902
Train  Accuracy: 96.7%
Test   Loss: 0.67174
Test   Accuracy: 56.7%
Epoch 10
Train  Loss: 0.17774
Train  Acc

In [ ]:

import mne
import numpy as np
from mne.io import concatenate_raws
from braindecode.datautil.signal_target import SignalAndTarget

physionet_paths = [ mne.datasets.eegbci.load_data(sub_id,[4,8,12,]) for sub_id in range(1,51)]
physionet_paths = np.concatenate(physionet_paths)
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths]

raw = concatenate_raws(parts)

picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=picks,
                baseline=None, preload=True)

physionet_paths_test = [mne.datasets.eegbci.load_data(sub_id,[4,8,12,]) for sub_id in range(51,56)]
physionet_paths_test = np.concatenate(physionet_paths_test)
parts_test = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths_test]
raw_test = concatenate_raws(parts_test)

picks_test = mne.pick_types(raw_test.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

events_test = mne.find_events(raw_test, shortest_event=0, stim_channel='STI 014')

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched_test = mne.Epochs(raw_test, events_test, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=picks_test,
                baseline=None, preload=True)

train_X = (epoched.get_data() * 1e6).astype(np.float32)
train_y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1
test_X = (epoched_test.get_data() * 1e6).astype(np.float32)
test_y = (epoched_test.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1
train_set = SignalAndTarget(train_X, y=train_y)
test_set = SignalAndTarget(test_X, y=test_y)